# Hessen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Hessen is `DE7`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [ ]:
import pandas as pd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
import zipfile
from datetime import datetime as dt
from io import StringIO
import warnings

from camelsp import Bundesland

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [ ]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Hessen').input_path
BASE

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [ ]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path,'raw_data', 'Pegel_für_Camels.xlsx')
    meta = pd.read_excel(path)
    return meta

# test it here
metadata = read_meta(BASE)

metadata

In [ ]:
# the id column will be Messstellen Nr.
id_column = 'Messstellen Nr.'

## file extract and parse

Should be straightforward, really nice data

In [ ]:
def extract_file(nr: Union[int, str], variable: str, input_path: str) -> pd.DataFrame:

    # build the path to the correct subfolder:
    path = os.path.join(input_path, variable, f"{nr}_{variable.upper()}.txt")

    # check file
    if not os.path.exists(path):
        return pd.DataFrame(columns=['date', variable.lower(), 'flag'])
    
    # read
    raw = pd.read_csv(path, skiprows=4, encoding='latin1', sep=' ', header=None)
    return pd.DataFrame({
        'date': [dt.strptime(str(_)[:8], '%Y%m%d') for _ in raw.iloc[:, 0]],
        variable.lower(): raw.iloc[:, 1],
        'flag': np.nan
    })

    return raw

extract_file(42, 'q', BASE)

### Finally run

Now, the Q and W data can be extracted along with the metadata. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [ ]:
with Bundesland('Hessen') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for provider_id in tqdm(metadata[id_column].values.astype(str)):
            # get q
            q = extract_file(provider_id, 'q', bl.input_path)
            w = extract_file(provider_id, 'w', bl.input_path)

            bl.save_timeseries(q, provider_id)
            bl.save_timeseries(w, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")
